<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/Transformers/AG_news_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention mechanism and Transformers

Task: AG news classifiction

## Imports

In [ ]:
!pip install pymorphy2 -q
!pip install datasets -q

In [ ]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pymorphy2
import re
import random
import seaborn as sns
import string
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import datasets
from datasets import load_dataset

import nltk
from nltk.tokenize import word_tokenize

from collections import Counter
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from transformers import get_scheduler
from typing import List

sns.set(palette='summer')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

### Create tensorboard

In [ ]:
%load_ext tensorboard

log_dir = "./logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
writer = SummaryWriter(log_dir)
%tensorboard --logdir {log_dir}

## Set config

In [1]:
@dataclass
class CFG:
  bert_model = 'bert-base-cased'
  custom_name = 'model_v0'
  batch_size = 32
  lr = 1e-5
  attention_head = 1
  train_size = 20000
  test_size = 1000
  padding_idx = 0
  hdden_dim = 768
  scheduler_lambda = 0.96

'bert-base-cased'

In [ ]:
STOP_WORDS = set(stopwords.words('english'))
PUNCT_TOKENIZER = nltk.WordPunctTokenizer()
MORPH = pymorphy2.MorphAnalyzer()

### Freeze seeds for reproducibility

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
seed_everything()

## Data preparation

In [ ]:
dataset = datasets.load_dataset('ag_news')